In [ ]:
# ProjectTwoDashboard.ipynb — CS-340 Project Two 
# Developed by Sharif Ayesh

from jupyter_dash import JupyterDash
from dash import html, dcc, dash_table, Input, Output
import dash_leaflet as dl
import pandas as pd
import os
from typing import Optional
from animal_shelter import AnimalShelter  #  CRUD class


# CONFIG
APP_TITLE = "Grazioso Salvare - Austin Animal Outcomes"
UNIQUE_ID = "Developed by Sharif Ayesh"

# Instantiate CRUD  
shelter = AnimalShelter()


# DATA HELPERS
def query_df(q: dict, limit: int = 1000):
    """Use your AnimalShelter.read() (returns list) and normalize to a DataFrame."""
    data = shelter.read(q) or []      # read() returns a list in your implementation
    if limit is not None:
        data = data[:limit]           # slice list instead of .limit()
    if not data:
        return pd.DataFrame()
    df = pd.json_normalize(data)
    preferred_cols = [
        'animal_id','name','breed','age_upon_outcome',
        'age_upon_outcome_in_weeks','sex_upon_outcome','outcome_type',
        'outcome_subtype','outcome_month','outcome_year',
        'location_lat','location_long'
    ]
    cols = [c for c in preferred_cols if c in df.columns]
    return df[cols] if cols else df

base_df = query_df({})
all_breeds = sorted(base_df['breed'].dropna().unique()) if 'breed' in base_df else []


# RESCUE TYPE FILTERS
RESCUE_QUERIES = {
    "Water Rescue": {
        "breed": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"],
        "sex_upon_outcome": ["Intact Female"],
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    },
    "Mountain/Wilderness Rescue": {
        "breed": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog",
                  "Siberian Husky", "Rottweiler"],
        "sex_upon_outcome": ["Intact Male"],
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    },
    "Disaster/Individual Tracking": {
        "breed": ["Doberman Pinscher", "German Shepherd", "Golden Retriever",
                  "Bloodhound", "Rottweiler"],
        "sex_upon_outcome": ["Intact Male"],
        "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
    }
}
RESCUE_TYPES = list(RESCUE_QUERIES.keys()) + ["Reset"]

def build_query(rescue_type: str, breed: Optional[str] = None):

    """Build Mongo query dict based on selected rescue type and optional breed."""
    if rescue_type and rescue_type != "Reset":
        spec = RESCUE_QUERIES[rescue_type]
        q = {"$and": [
            {"breed": {"$in": spec["breed"]}},
            {"sex_upon_outcome": {"$in": spec["sex_upon_outcome"]}},
            {"age_upon_outcome_in_weeks": spec["age_upon_outcome_in_weeks"]}
        ]}
    else:
        q = {}
    if breed and breed != "Any":
        if q == {}:
            q = {"breed": breed}
        else:
            q["$and"].append({"breed": breed})
    return q


# DASH APP
app = JupyterDash(__name__)
app.title = APP_TITLE

# Put grazioso_logo.png 
logo_link = html.A(
    html.Img(src="assets/grazioso_logo.png", style={"height":"60px"}),
    href="https://www.snhu.edu", target="_blank", rel="noopener noreferrer"
)

app.layout = html.Div([
    # Header
    html.Div([
        html.Div([logo_link], style={"display":"inline-block","verticalAlign":"middle","marginRight":"16px"}),
        html.Div([
            html.H2(APP_TITLE, style={"margin":"0"}),
            html.P(UNIQUE_ID, style={"margin":"0","opacity":0.7})
        ], style={"display":"inline-block","verticalAlign":"middle"})
    ], style={"marginBottom":"16px"}),

    # Controls
    html.Div([
        html.Div([
            html.Label("Rescue Type"),
            dcc.RadioItems(
                id="rescue-type",
                options=[{"label": r, "value": r} for r in RESCUE_TYPES],
                value="Reset"
            ),
        ], style={"flex":"1","minWidth":"240px","paddingRight":"16px"}),

        html.Div([
            html.Label("Preferred Breed"),
            dcc.Dropdown(
                id="breed-dd",
                options=[{"label":"Any","value":"Any"}] + [{"label":b,"value":b} for b in all_breeds],
                value="Any", searchable=True, clearable=False
            )
        ], style={"flex":"1","minWidth":"280px","paddingRight":"16px"}),

        html.Div([
            html.Button("Reset Filters", id="reset-btn", n_clicks=0, style={"marginTop":"24px"})
        ])
    ], style={"display":"flex","flexWrap":"wrap","marginBottom":"12px"}),

    # Data table
    dash_table.DataTable(
        id="animal-table",
        columns=[{"name": c, "id": c} for c in base_df.columns],
        data=base_df.to_dict("records"),
        page_size=10,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        style_table={"overflowX":"auto"},
        style_cell={"textAlign":"left","padding":"6px"},
        style_header={"fontWeight":"bold"}
    ),

    html.Br(),

    # Charts / Map
    html.Div([
        html.Div([
            html.H4("Outcome Types"),
            dcc.Graph(id="outcome-pie")
        ], style={"flex":"1","minWidth":"320px","paddingRight":"16px"}),

        html.Div([
            html.H4("Geolocation Map"),
            dl.Map(center=[30.27, -97.74], zoom=9, id="geo-map", children=[
                dl.TileLayer(),
                dl.LayerGroup(id="marker-layer")
            ], style={"width":"100%","height":"420px"})
        ], style={"flex":"1","minWidth":"320px"})
    ], style={"display":"flex","flexWrap":"wrap"})
], style={"maxWidth":"1200px","margin":"0 auto","padding":"16px"})


# CALLBACKS
@app.callback(
    Output("rescue-type","value"),
    Output("breed-dd","value"),
    Input("reset-btn","n_clicks"),
    prevent_initial_call=True
)
def on_reset(n):
    return "Reset", "Any"

@app.callback(
    Output("animal-table","data"),
    Input("rescue-type","value"),
    Input("breed-dd","value")
)
def update_table(rescue_type, breed):
    q = build_query(rescue_type, breed)
    df = query_df(q)
    return df.to_dict("records")

@app.callback(
    Output("outcome-pie","figure"),
    Output("marker-layer","children"),
    Input("animal-table","data")
)
def update_visuals(rows):
    df = pd.DataFrame(rows)

    # Pie chart
    if not df.empty and "outcome_type" in df:
        pie_df = df["outcome_type"].fillna("Unknown").value_counts().reset_index()
        pie_df.columns = ["outcome_type","count"]
        fig = {
            "data": [{
                "type":"pie",
                "labels": pie_df["outcome_type"],
                "values": pie_df["count"],
                "hoverinfo":"label+percent",
                "textinfo":"value"
            }],
            "layout": {"margin":{"l":0,"r":0,"t":0,"b":0}}
        }
    else:
        fig = {"data": [], "layout": {"title":"No data"}}

    # Map markers for ALL animals in filtered dataset
    markers = []
    if not df.empty and {"location_lat","location_long"}.issubset(df.columns):
        for _, row in df.dropna(subset=["location_lat","location_long"]).iterrows():
            markers.append(
                dl.Marker(
                    position=[row["location_lat"], row["location_long"]],
                    children=dl.Tooltip(row.get("breed","Unknown"))
                )
            )
    return fig, markers


# RUN
app.run_server(mode="inline", debug=False)
